#####Explicação


A correspondência sequencial é necessária para evitar que o algoritmo de busca percorra todas as palavras para cada padrão. A correspondência está configurada para que as palavras sejam buscadas pelos maiores padrões e, havendo palavras restantes, a busca continua com os padrões menores na mesma sequencia. Este método garante que as maiores expressões sejam capturadas por inteiro e que não haja segmentação das expressões resultando em erros de classificação.


## BUSCA SEQUENCIAL DE PADRÕES

In [ ]:
import spacy
from spacy.matcher import Matcher

In [ ]:
!python -m spacy download pt_core_news_md

In [ ]:
#Carrega o texto do repositório e o converte para uma lista de sentenças
import requests
#url = "https://raw.githubusercontent.com/dglopes/NBR15575/main/RequisitosP1_B_v4.txt"
url = "https://github.com/dglopes/NBR15575/raw/main/Cap8_SegurancaIncendio_v1.txt"

texto_completo = requests.get(url).text
texto = texto_completo.split("\r\n")
print("total de sentenças:", len(texto),"\n")

for t in range(len(texto)):
    print(f"{t+1} {texto[t]}" )

In [ ]:
#Dicionário com padrões organizados por rótulo
padroes = [
    {"label": "REFERENCIA", "pattern": [
        [{"LOWER": "normas"},{"POS": "ADJ"}],#modif
        [{"LOWER": "normas"},{"LOWER": "específicas"}],#modif
        [{"LOWER": "normas"},{"LOWER": "brasileiras"}],#modif
        [{"LOWER": "legislação"},{"LOWER": "vigente"}],#modif
        [{"LOWER": "abnt"},{"LOWER": "nbr"},{"POS":"NUM"}],
        [{"POS": "PROPN", "MORPH": {"IS_SUPERSET": ["Gender=Fem"]}},{"POS": "PROPN"},{"POS": "NUM"}],
        [{"POS": "PROPN", "MORPH": {"IS_SUPERSET": ["Gender=Fem"]}},{"POS": "ADP"},{"POS": "NUM"}],
        [{"POS": "PROPN", "MORPH": {"IS_SUPERSET": ["Gender=Fem"]}},{"POS": "ADP"},{"POS": "NUM"}],
        [{"POS": "NOUN", "MORPH": {"IS_SUPERSET": ["Gender=Fem"]}},{"POS": "PROPN"},{"POS": "ADJ"}],
        [{"POS": "PROPN", "DEP":"obj"},{"POS": "PROPN"}]
    ]},

    {"label": "EXIGENCIA", "pattern": [
        [{"POS": "VERB"},{"POS": "AUX"},{"POS": "VERB"},{"POS": "CCONJ"}, {"POS": "VERB"}],#deve ser projetadas e executadas
        [{"POS": "VERB"},{"POS": "AUX"},{"POS": "CCONJ"}, {"POS": "VERB"}],
        [{"POS": "VERB"},{"POS": "ADV", "OP": "?"},{"POS": "AUX"},{"POS": "VERB"}], #deve ser projetada; deve ser atendida
        [{"POS": "VERB"},{"POS": "VERB"},{"POS": "ADP"}], #deve ser feita;
        [{"LOWER": "não", "OP": "?"},{"POS": "VERB"},{"POS": "VERB"}], # (não)deve dispor; (não)deve atender; deve possibilitar; deve ter
        [{"LOWER": "não", "OP": "?"},{"POS": "VERB"},{"POS": "VERB"}],
        [{"LOWER": "não", "OP": "?"},{"POS": "VERB"},{"POS": "AUX"},{"POS": "ADJ"}] #(não) pode ser menor
        ]},

    {"label": "COMPONENTE", "pattern": [
        [{"POS": "NOUN"},{"POS": "ADP"},{"POS": "NOUN"}],
        [{"POS": "NOUN"},{"POS": "ADP"},{"POS": "NOUN"},{"POS": "ADJ"}],#modif
        [{"POS": "NOUN"},{"POS": "ADP"},{"POS": "ADJ"}],
        [{"POS": "NOUN", "DEP":"nsubj"},{"POS": "ADJ"},{"POS": "ADJ"}],
        [{"POS": "NOUN"},{"POS": "ADP"},{"POS": "NOUN"},{"POS": "ADP"},{"POS": "NOUN"}],#modif
        [{"POS": "NOUN"},{"POS": "ADP"},{"POS": "NOUN"},{"POS": "ADP"},{"POS": "NOUN"},{"POS": "VERB"},{"POS": "PUNCT"}],#modif
        [{"POS": "NOUN", "DEP":"nsubj"}],
        [{"POS": "NOUN"},{"POS": "ADJ"}]
    ]},

    {"label": "PARAMETRO", "pattern": [
        [{"POS": "NOUN"}, {"POS": "ADV"},{"POS": "ADP"},{"POS": "NOUN"}],
        [{"POS": "NOUN", "DEP":"obj"}, {"POS": "ADP"},{"POS": "NOUN"}]
    ]},

    {"label": "VALOR", "pattern": [
        [{"POS":"NUM"},{"POS": "NOUN"}],
        [{"POS":"NUM"},{"POS": "SYM"}]
    ]}
]

In [ ]:
#Funcão para buscar os padrões de modo sequencial
from spacy.matcher import Matcher
from spacy.tokens import Span

def buscar_padroes_sequencialmente(doc, padroes):
    resultados = []
    tokens_processados = set()

    for padrao in padroes:
        label = padrao["label"]
        matcher = Matcher(doc.vocab)

        for i, padrao_atual in enumerate(padrao["pattern"]):
            matcher.add(f"{label}", [padrao_atual], greedy = "LONGEST")

        for padrao_id, inicio, fim in matcher(doc):
            rotulo = matcher.vocab.strings[padrao_id]

            # Verifica se todos tokens já foram processados
            if any(token.i in tokens_processados for token in doc[inicio:fim]):
                continue

            # Adiciona os tokens do padrão ao conjunto de tokens processados
            tokens_processados.update(token.i for token in doc[inicio:fim])

            # Converte os tokens correspondentes em um Span e adiciona à lista
            span = Span(doc, inicio, fim, label=rotulo)
            resultados.append((rótulo, span))

    return resultados


In [ ]:
nlp = spacy.load("pt_core_news_md")

lista_resultados = []

for i, sentence in enumerate(texto,1):
    doc = nlp(sentence)

    resultados = buscar_padroes_sequencialmente(doc, padroes) #aqui carrega a função de busca de padrões

    resultados_sentenca = {
        "FRASE" : sentence,
        "TERMOS" : []
    }

    for rotulo, span in resultados:
        resultados_sentenca["TERMOS"].append({
            "ROTULO" : rotulo,
            "TEXTO" : span.text
        })

    lista_resultados.append(resultados_sentenca)

In [ ]:
for i, item in enumerate(lista_resultados,1):
    print(i, item)

##### explicacao

O resultado é uma lista armazenada na variável lista_resultados. Mas esta lista é uma lista de dicionários. Cada item da lista é um dicionário composto pelas chaves ["FRASE"] e ["TERMOS"]. Já termos armazena outra lista de dicionários contendo as chaves ["ROTULO"] e ["TEXTO"].

####Gerar XML

In [ ]:
import xml.etree.ElementTree as ET

dados = lista_resultados

root = ET.Element("ROOT")

for dado in dados:
    frase_element = ET.SubElement(root, "FRASE")
    frase_element.text = dado['FRASE']

    termos_element = ET.SubElement(frase_element, "TERMOS")

    for termo in dado['TERMOS']:
        termo_element = ET.SubElement(termos_element, termo['ROTULO'])
        termo_element.text = termo['TEXTO']

tree = ET.ElementTree(root)
tree.write("output_v2.xml")
